<a href="https://colab.research.google.com/github/stxupengyu/Milti-Label-Classification-Data-Preprocessing/blob/master/Label2Glove_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#挂载云盘 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#进入项目目录
print('original document')
!ls
import os
os.chdir('/content/drive/My Drive/colab/XML')
print('current document')
!ls

original document
EURLex.py	   glove.6B.50d.txt	      rcv_embedding.npz
glove.6B.100d.txt  glove.6B.zip		      XML-Glove-Embedding.ipynb
glove.6B.200d.txt  rcv1.topics.hier.orig.txt
glove.6B.300d.txt  rcv_embedding_mean.npz
current document
EURLex.py	   glove.6B.50d.txt	      rcv_embedding.npz
glove.6B.100d.txt  glove.6B.zip		      XML-Glove-Embedding.ipynb
glove.6B.200d.txt  rcv1.topics.hier.orig.txt
glove.6B.300d.txt  rcv_embedding_mean.npz


In [ ]:
#下载glove
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!ls
#!unzip glove.6B.zip
import numpy as np
import re
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords 

glove_dim=300
data_path='rcv1.topics.hier.orig.txt'

def body_remove_sw(sentence):
    #去除stopwords 并把句子转换成单词的列表
    words= [w for w in sentence.split(' ') if w not in sw]
    return words
def body_wnl(words):
    #做词形还原
    words=[wnl.lemmatize(word) for word in words]
    return words
def preprocessing(sentence,del_estr):
  #处理每个原始标签
    sentence=sentence.lower()#字母小写化
    replace = " "*len(del_estr)
    tran_tab = str.maketrans(del_estr, replace)
    sentence = sentence.translate(tran_tab)#完成上述去除标点符号的功能
    words=sentence.split(' ')#根据空格把句子分隔成单词
    words=body_remove_sw(sentence)#去除停用词
    while '' in words:
      words.remove('')#去除为空格的空值
    words=body_wnl(words)#词性还原
    return words
def read_data(data_path):
  #读取原始数据，得到处理好的标签列表
    iFile = open(data_path, "r")
    label=[]
    wnl = WordNetLemmatizer() 
    sw=stopwords.words('english')#这两步都是在循环前调用为了加快速度
    del_estr = string.punctuation + string.digits+string.whitespace  # 去除ASCII 标点符号，数字
    for line in iFile.readlines():
        lineVec = line.strip().split(":")#根据:将一行分割
        label_i=lineVec[-1]
        label_i=preprocessing(label_i,del_estr)
        label.append(label_i)
    return label 
label=read_data(data_path)[1:]
#label#展示一下处理好的label

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def sum_tag_words(label):
  #统计所有标签词
  sum_tag=[]
  for label_i in label:
    for i in label_i:
      if i not in sum_tag:
        sum_tag.append(i)
  return sum_tag
def glove_dict(glove_dim):
  #构建一个当前标签词在glove的embedding字典
  all_tag_words=sum_tag_words(label)
  print('使用的是 glove embedding dim=%d'%glove_dim)
  #所有的标签词的list，用于从glove中抽取词表示
  iFile = open("glove/glove.6B.%dd.txt"%glove_dim, "r")#有119万个词
  gloveDict = {}#glove word 对应id的字典
  count=0
  for line in iFile.readlines():
      lineVec = line.strip().split(" ")#将一行转化为一个列表
      if lineVec[0] in all_tag_words:
        gloveDict[lineVec[0]]=lineVec[1:]
  print('%d个tag词有%d有glove embedding'%(len(all_tag_words),len(gloveDict)))
  return gloveDict
def str2float(str_list):
  #将glove的词向量权重从str转化为float np.array
  record=[]
  for i in str_list:
    record.append(float(i))
  return np.array(record)
def many2one(label_embedding_i):
  #将多个词的embediding合并为二维array，并取均值
  label_embedding=label_embedding_i[0]
  for inst in label_embedding_i[1:]:
    label_embedding=np.vstack((label_embedding,inst))
  if len(label_embedding.shape)>1:
    label_embedding=np.mean(label_embedding,axis=0)#均值操作
  return label_embedding
def glove_embedding(label):
  #将原始标签表示为embedding
  label_embedding_list=[]
  for label_i in label:
    label_embedding_i=[]
    for word in label_i:
      label_embedding_i.append(str2float(glovedict[word]))
    label_embedding_i=many2one(label_embedding_i)
    label_embedding_list.append(label_embedding_i)
  return label_embedding_list
glovedict=glove_dict(glove_dim)
label_embedding=glove_embedding(label)

使用的是 glove embedding dim=300
139个tag词有139有glove embedding


In [ ]:
file_name='rcv_embedding_mean.npz'
#file_name='rcv_embedding.npz'
def save_data(label_embedding):
  label_embedding=np.array(label_embedding)
  np.savez(file_name,label_embedding=label_embedding,allow_pickle=True)
  print('数据集已经保存成功在文件%s中！'%(file_name))
def load_data():
  data=np.load(file_name,allow_pickle=True)
  label_embedding=data['label_embedding']
  return label_embedding
save_data(label_embedding)

数据集已经保存成功在文件rcv_embedding_mean.npz中！


In [ ]:
label_embedding=load_data()

In [ ]:
label_embedding.shape

(103, 300)